# Lecture 10
_____________


Import libraries

In [28]:
import numpy as np 
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 600
from scipy import optimize

## _NLP examples_

Non-linear programming examples for a simple constraint problem

2-user water allocation example

_First approach:_ `scipy.optimize.minimize`

__Objective function__ (x is decision variable vector)

In [29]:
def f(x):
    return -1*(30*x[0] - 4*x[0]**2 + 10*x[1] - 2*x[1]**2) # quadratic benefit function (maximize)

If constraint function returns > zero, constraint satisfied. otherwise infeasible

In [30]:
def f_constraint(x):
    return -(x[0] + x[1] - 5)

In [31]:
constraint = {'type': 'ineq', 'fun': f_constraint}

sol = optimize.minimize(f, x0 = [0,5], constraints=[constraint])
print(sol) # does not return duals

     fun: -66.66666666666687
     jac: array([-3.33333302, -3.33333397])
 message: 'Optimization terminated successfully'
    nfev: 7
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([3.33333333, 1.66666667])


Decision variables `sol.x`, objective function `sol.fun`

In [32]:
import cvxpy as cvx

In [33]:
x1 = cvx.Variable()
x2 = cvx.Variable()
Q = 5 # units of water

obj = cvx.Maximize(30*x1 - 4*x1**2 + 10*x2 - 2*x2**2)

constraints = [x1 + x2 <= Q, x1 >= 0, x2 >= 0] # magic?

prob = cvx.Problem(obj, constraints)
prob.solve()

print('Objective = %f' % obj.value)
print('X1 = %f' % x1.value)
print('X2 = %f' % x2.value)

for c in constraints:
  print('Dual (%s) = %f' % (c, c.dual_value))

Objective = 66.666667
X1 = 3.333333
X2 = 1.666667
Dual (var271 + var272 <= 5.0) = 3.333333
Dual (0.0 <= var271) = 0.000000
Dual (0.0 <= var272) = 0.000000


------------

## _LP farm example_

Linear programming examples for crop choice irrigation example problem

_First approach:_ `scipy.optimize.linprog`

Need matrix form: minimize c^T * x, subject to Ax <= b

In [34]:
c = [-5, -3] # negative to maximize (b/c the default is minimize)
A = [[10,5], [1,1.5], [2,2], [-1,0], [0,-1]]
b = [20, 3, 4.5, 0, 0]

In [35]:
sol = optimize.linprog(c, A, b)
print('Scipy Output:')
print(sol)

Scipy Output:
     con: array([], dtype=float64)
     fun: -10.249999999652978
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([ 8.90629792e-10,  5.00000000e-01, -1.84616766e-11,  1.75000000e+00,
        5.00000000e-01])
  status: 0
 success: True
       x: array([1.75, 0.5 ])


^ Look at x value: number of crops..?

_Second approach:_ `cvxpy`


In [36]:
xc = cvx.Variable(name='xc')
xb = cvx.Variable(name='xb')
pc = 5
pb = 3

obj = cvx.Maximize(pc*xc + pb*xb)

constraints = [10*xc + 5*xb <= 20, # cvxpy recognizes the logical comparison signs
               xc + 1.5*xb <= 3,
               2*xc + 2*xb <= 4.5,
               xc >= 0,
               xb >= 0]

prob = cvx.Problem(obj, constraints)
prob.solve()

10.249999998189105

In [37]:
print('\ncvxpy Output:')
print('Objective = %f' % obj.value)
print('xc = %f' % xc.value)
print('xb = %f' % xb.value)

for c in constraints:
    print('Dual (%s) = %f' % (c, c.dual_value))


cvxpy Output:
Objective = 10.250000
xc = 1.750000
xb = 0.500000
Dual (10.0 @ xc + 5.0 @ xb <= 20.0) = 0.400000
Dual (xc + 1.5 @ xb <= 3.0) = 0.000000
Dual (2.0 @ xc + 2.0 @ xb <= 4.5) = 0.500000
Dual (0.0 <= xc) = 0.000000
Dual (0.0 <= xb) = 0.000000


Should match the answer above

The units of the dual values come from the objective function and constraint value ($ / constraint value)

Constraints with nonzero dual values are binding

-----------

#### Interactive widget

In [40]:
def lpplot(pc=5, pb=3, W=20, L=3, F=4.5):
    A = np.array([[10,1,2,-1,0], [5,1.5,2,0,-1 ]]).T
    b = np.array([ W, L, F, 0, 0])
    c = np.array([ -pc, -pb ])
    sol = optimize.linprog(c,A,b)

    x = np.arange(0,5.01,0.01)
    X1,X2 = np.meshgrid(x,x)
    Z = -c[0]*X1 + -c[1]*X2

    # Contour lines in the background
    plt.contour(X1,X2,Z,50,cmap=plt.cm.cool)
    plt.colorbar()

    # Plot constraint lines
    plt.plot(x, (W-10*x)/5, color='k')
    plt.plot(x, (L-x)/1.5, color='k')
    plt.plot(x, (F-2*x)/2, color='k')

    # And the optimal point ...
    plt.scatter(sol['x'][0], sol['x'][1], s=40, color='r', zorder=5)
    plt.xlim([0,3])
    plt.ylim([0,3])
    plt.xlabel('$x_c$')
    plt.ylabel('$x_b$')
    plt.show()



In [42]:
from ipywidgets import interact

interact(lpplot, pc=(0.,12.), pb=(0.,12.), W=(15.,25.), L=(1.,5.), F=(3.,6.))

interactive(children=(FloatSlider(value=5.0, description='pc', max=12.0), FloatSlider(value=3.0, description='…

<function __main__.lpplot(pc=5, pb=3, W=20, L=3, F=4.5)>